# Kapitel 13 - 03

> ### <span style="color:red">ACHTUNG:</span> Bitte zum Starten im Menü `Cell->Run All` ausführen.

## Deaktivieren der Warnungen

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Cats and Dogs

#### Listing 13.19

In [ ]:
import pandas as pd
import requests
import os
import PIL
import shutil
import numpy as np


#### Listing 13.20

In [ ]:
basedir = r'cat_dog'
dir_dict = {
    'train':  os.path.join(basedir,'training_set'),
    'test':  os.path.join(basedir,'test_set'),
    'train_cats': os.path.join(basedir,'training_set','cats'),
    'train_dogs': os.path.join(basedir,'training_set','dogs'),
    'test_cats': os.path.join(basedir,'test_set','cats'),
    'test_dogs': os.path.join(basedir,'test_set','dogs')
           }

# Erzeuge die Verzeichnisse (nur wenn sie noch nicht vorhanden sind)
for key in dir_dict:
    if not os.path.exists(dir_dict[key]): 
        os.makedirs(dir_dict[key])

#### Listing 13.21

In [ ]:
print("Anzahl Katzenbilder (Training):",len(os.listdir(dir_dict['train_cats'])))
print("Anzahl Hundebilder (Training):",len(os.listdir(dir_dict['train_dogs'])))
print("Anzahl Katzenbilder (Test):",len(os.listdir(dir_dict['test_cats'])))
print("Anzahl Hundebilder (Test):",len(os.listdir(dir_dict['test_dogs'])))

#### Listing 13.22

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

model = models.Sequential()
base_model = InceptionV3(weights='imagenet',include_top=False,input_shape=(200, 200, 3))
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#### Listing 13.23

In [ ]:
base_model.trainable = False
model.summary()

#### Listing 13.24

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

#### Listing 13.25

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(        
        dir_dict['train'],      
        target_size=(200, 200),
        batch_size=20,        
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        dir_dict['test'],
        target_size=(200, 200),
        batch_size=20,
        class_mode='binary')

#### Listing 13.26

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=40,
      validation_data=validation_generator,
      validation_steps=50)

#### Listing 13.27

In [ ]:
model.save(os.path.join(basedir,'cats_dogs_1.h5'))

#### Listing zur Erstelllung der Graphen

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training Genauigkeit')
plt.plot(epochs, val_acc, 'b', label='Validierung Genauigkeit')
plt.title('Training und Validierung Genauigkeit')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validierung loss')
plt.title('Training und Validierung Loss')
plt.legend()

plt.show()